In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

sp500 = pd.read_csv('sp500.csv')
symbols = sp500['Symbol'].tolist()

start_date = '1990-12-31'
end_date   = '2025-04-30'
all_tickers = symbols + ['SPY']

monthly = yf.download(
    tickers=all_tickers,
    start=start_date,
    end=end_date,
    interval='1mo',
    group_by='ticker',
    auto_adjust=True,
    threads=True,
    progress=False
)

available = [sym for sym in symbols if sym in monthly.columns.get_level_values(0)]
missing   = sorted(set(symbols) - set(available))
if missing:
    print(f"{missing} missing")

price_m = pd.DataFrame({ sym: monthly[sym]['Close'] for sym in available })
returns = price_m.pct_change().dropna(how='all')

daily = yf.download(
    tickers=all_tickers,
    start=start_date,
    end=end_date,
    interval='1d',
    group_by='ticker',
    auto_adjust=True,
    threads=True,
    progress=True
)

price_d = pd.DataFrame({ sym: daily[sym]['Close'] for sym in available })

spy_ret = monthly['SPY']['Close'].pct_change().dropna()

window = 60
risk = {}
for sym in available:
    r = returns[sym]
    risk[(sym,'60VOL')] = r.rolling(window).std()
    cov = r.rolling(window).cov(spy_ret)
    var_spy = spy_ret.rolling(window).var()
    risk[(sym,'BETA')] = cov / var_spy
    risk[(sym,'SKEW')] = r.rolling(window).skew()

risk_df = pd.DataFrame(risk)
risk_df.columns.names = ['Symbol','Descriptor']
price_d.to_csv('daily_prices_sp500.csv')
returns.to_csv('monthly_returns_sp500.csv')
risk_df.to_csv('risk_descriptors_sp500.csv')

print("Done")



In [17]:
!pip -q install innvestigate tensorflow

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm

# PERIOD AND WINDOW SETTINGS
START_PERIOD = pd.Period("2012-01", "M")
END_PERIOD   = pd.Period("2017-12", "M")

descriptor_window = 12
TRAIN_MONTHS      = 12
SEQ_LEN           = 5
VAL_SPLIT         = 1/12

sp500   = pd.read_csv("/content/sp500.csv")
tickers = sp500["Symbol"].tolist()

ret_df   = pd.read_csv("/content/monthly_returns_sp500.csv",
                       parse_dates=["Date"], index_col="Date")[tickers]
raw_desc = (pd.read_csv("/content/risk_descriptors_sp500.csv",
                        header=[0, 1], index_col=0, parse_dates=True)[tickers])
raw_desc.columns.names = ["Ticker", "Descriptor"]


# align indices to periods and cut to common range
ret_df.index, raw_desc.index = ret_df.index.to_period("M"), raw_desc.index.to_period("M")
min_period = pd.Period("2010-01", "M")
ret_df, raw_desc = ret_df.loc[ret_df.index >= min_period], raw_desc.loc[raw_desc.index >= min_period]
common = ret_df.index.intersection(raw_desc.index)
ret_df, raw_desc = ret_df.loc[common], raw_desc.loc[common]


# BUILD FEATURE AND TARGET ARRAYS
descriptors = raw_desc.columns.get_level_values("Descriptor").unique().tolist()
N_FEAT = len(descriptors)

dates, S = common, len(tickers)
F = np.empty((len(dates), S, N_FEAT), dtype=float)
for j, tkr in enumerate(tickers):
    F[:, j, :] = raw_desc.xs(tkr, level="Ticker", axis=1).values

R = ret_df[tickers].values
Y = np.roll(R, -1, axis=0)

# DEFINE MODELS
after_full = dates[descriptor_window:-1]
after_idx  = [p for p in after_full if START_PERIOD <= p <= END_PERIOD]
period_to_idx = {p: i for i, p in enumerate(dates)}

lin_reg = make_pipeline(StandardScaler(), LinearRegression())
svr     = make_pipeline(StandardScaler(), LinearSVR(max_iter=10000))
rf      = RandomForestRegressor(n_estimators=50, max_depth=8, n_jobs=-1, random_state=0)

tf.random.set_seed(0)
dnn  = Sequential([Input(shape=(SEQ_LEN * N_FEAT,)),
                   Dense(40, activation="relu"), Dense(1)])
dnn.compile("adam", "mse")

lstm = Sequential([Input(shape=(SEQ_LEN, N_FEAT)),
                   Bidirectional(LSTM(16)), Dense(1)])
lstm.compile("adam", "mse")


# LRP SCORING FUNCTION
def lrp_scores(model, X, is_lstm=False):
    X_tf = tf.convert_to_tensor(X)
    with tf.GradientTape() as tape:
        tape.watch(X_tf)
        y_pred = model(X_tf, training=False)
    grads = tape.gradient(y_pred, X_tf)
    gxi = tf.abs(grads * X_tf)
    return (tf.reduce_sum(gxi, axis=[1, 2]) if is_lstm
            else tf.reduce_sum(gxi, axis=1)).numpy()

# SETUP METRIC CONTAINERS & EVAL PERIODS
models = ["LSTM", "LSTM+LRP", "DNN", "DNN+LRP", "Linear", "LinSVR", "RF"]
mae_hist, rmse_hist, port_hist = ({m: [] for m in models} for _ in range(3))


#  Determine which months to evaluate over
for today in tqdm(after_idx, desc="Eval 2012-2017"):
    i_today = period_to_idx[today]
    train_idx = [i for i in range(i_today - TRAIN_MONTHS, i_today)
                 if i >= descriptor_window]
    if not train_idx:
        continue

    # BUILD TRAINING SET
    Xtr_rows, ytr_rows = [], []
    for i_d in train_idx:
        xb = F[i_d - SEQ_LEN + 1:i_d + 1].transpose(1, 0, 2).reshape(S, SEQ_LEN * N_FEAT)
        yb = Y[i_d]
        mask = (~np.isnan(yb)) & (~np.isnan(xb).any(axis=1))
        if mask.any():
            Xtr_rows.append(xb[mask]); ytr_rows.append(yb[mask])
    if not Xtr_rows:
        continue
    Xtr, ytr = np.vstack(Xtr_rows), np.hstack(ytr_rows)

    # BUILD TEST SET
    x_test = F[i_today - SEQ_LEN + 1:i_today + 1].transpose(1, 0, 2).reshape(S, SEQ_LEN * N_FEAT)
    y_test = Y[i_today]
    mask = (~np.isnan(y_test)) & (~np.isnan(x_test).any(axis=1))
    if not mask.any():
        continue
    Xte, yte = x_test[mask], y_test[mask]
    q = max(1, len(yte) // 5)

    # FIT MODELS & PREDICT
    p_lr  = lin_reg.fit(Xtr, ytr).predict(Xte)
    p_svr = svr.fit(Xtr, ytr).predict(Xte)
    rf.fit(Xtr, ytr); p_rf = rf.predict(Xte)

    dnn.fit(Xtr, ytr, validation_split=VAL_SPLIT, epochs=4, batch_size=512,
            verbose=0, callbacks=[EarlyStopping(patience=1,
                                                restore_best_weights=True)])
    p_dnn = dnn.predict(Xte, verbose=0).ravel()
    r_dnn = lrp_scores(dnn, Xte)

    Xtr_s, Xte_s = Xtr.reshape(-1, SEQ_LEN, N_FEAT), Xte.reshape(-1, SEQ_LEN, N_FEAT)
    lstm.fit(Xtr_s, ytr, validation_split=VAL_SPLIT, epochs=4, batch_size=256,
             verbose=0, callbacks=[EarlyStopping(patience=1,
                                                 restore_best_weights=True)])
    p_lstm = lstm.predict(Xte_s, verbose=0).ravel()
    r_lstm = lrp_scores(lstm, Xte_s, is_lstm=True)

    # Collect all predictions in a dict
    preds = {
        "LSTM": p_lstm,       "LSTM+LRP": r_lstm,
        "DNN":  p_dnn,        "DNN+LRP":  r_dnn,
        "Linear": p_lr,       "LinSVR":   p_svr,
        "RF": p_rf,
    }


    # RECORD METRICS & PORTFOLIO RETURNS
    for name, pred in preds.items():
        if name in ("LSTM", "DNN", "Linear", "LinSVR", "RF"):
            mae_hist[name].append(mean_absolute_error(yte, pred))
            rmse_hist[name].append(np.sqrt(mean_squared_error(yte, pred)))
        else:
            mae_hist[name].append(np.nan); rmse_hist[name].append(np.nan)

        idx = np.argsort(pred); longs, shorts = idx[-q:], idx[:q]
        port_hist[name].append(yte[longs].mean() - yte[shorts].mean())

# AGGREGATE RESULTS INTO SUMMARY TABLE
rows = []
for name in models:
    mae_vals  = np.asarray(mae_hist[name],  dtype=float)
    rmse_vals = np.asarray(rmse_hist[name], dtype=float)

    # Compute average MAE/RMSE
    mae  = np.nanmean(mae_vals)  if np.isfinite(mae_vals).any()  else np.nan
    rmse = np.nanmean(rmse_vals) if np.isfinite(rmse_vals).any() else np.nan


    # Compute annualized return, volatility, and Sharpe ratio
    rets = np.asarray(port_hist[name], dtype=float)
    if rets.size:
        ann_rt = (1 + rets).prod()**(12 / rets.size) - 1
        ann_vl = rets.std(ddof=0) * np.sqrt(12)
        sharpe = ann_rt / ann_vl if ann_vl else np.nan
    else:
        ann_rt = ann_vl = sharpe = np.nan

    rows.append(dict(Model=name, Return=ann_rt * 100, Vol=ann_vl * 100,
                     Sharpe=sharpe, MAE=mae, RMSE=rmse))

# Build and format the final summary DataFrame, then print
summary = (pd.DataFrame(rows)
             .set_index("Model")
             .rename(columns={"Return":"Return[%]", "Vol":"Vol[%]"}))

col_blocks = [
    ("Deep recurrent factor model", "LSTM"),
    ("Deep recurrent factor model", "LSTM+LRP"),
    ("Deep factor model",          "DNN"),
    ("Deep factor model",          "DNN+LRP"),
    ("Linear model",               "Linear"),
    ("SVR",                        "LinSVR"),
    ("Random forest",              "RF"),
]
summary = summary.reindex(columns=["Return[%]", "Vol[%]", "Sharpe", "MAE", "RMSE"])
summary = summary.T.reindex([m for _, m in col_blocks], axis=1)
summary.columns = pd.MultiIndex.from_tuples(col_blocks,
                                            names=["Family", "Method"])

fmt = {k: "{:.2f}" for k in ["Return[%]", "Vol[%]", "Sharpe"]}
fmt.update({k: "{:.5f}" for k in ["MAE", "RMSE"]})

pretty = summary.copy()
for row in pretty.index:
    pretty.loc[row] = pretty.loc[row].apply(
        lambda v, f=fmt[row]: f.format(v) if np.isfinite(v) else "—"
    )

print("\nSummary (2012-2017)")
print(pretty)


Eval 2012-2017: 100%|██████████| 72/72 [08:14<00:00,  6.87s/it]


Summary (2012-2017)
Family    Deep recurrent factor model          Deep factor model          \
Method                           LSTM LSTM+LRP               DNN DNN+LRP   
Return[%]                        5.70     5.79             -1.52    3.37   
Vol[%]                           9.00    13.33              6.41   10.49   
Sharpe                           0.63     0.43             -0.24    0.32   
MAE                           0.05011        —           0.05883       —   
RMSE                          0.06706        —           0.08032       —   

Family    Linear model      SVR Random forest  
Method          Linear   LinSVR            RF  
Return[%]         2.50     2.02          2.08  
Vol[%]            8.79     7.79          9.35  
Sharpe            0.28     0.26          0.22  
MAE            0.05073  0.05088       0.05050  
RMSE           0.06771  0.06791       0.06782  
